In [ ]:
print "Attendi..."
import numpy as np
import matplotlib.pyplot as plt
import os
from math import floor
from os import path

# parametri della simulazione
n_traces = 2 # da impostare
n_ues = 12
time_quantum = 0.0256 # da impostare
time_start = 0.0000 # da impostare
time_end = 60.0 # da impostare

# percorso di input
in_path = "/home/luca/Scaricati/output/prova/tp/" # da impostare

# impostare a "" (stringa vuota) per non caricare i dati dal file npz, altrimenti specificare il nome del file npz
in_file = "" # da impostare
load_from_npz = (in_file != "")

# inizializzazione
times = np.arange(time_start, time_end, time_quantum)
mean_throughput = np.zeros((n_ues, len(times)))
power_throughput = np.zeros((n_ues, len(times)))
mean_delay = np.zeros((n_ues, len(times)))
power_delay = np.zeros((n_ues, len(times)))
curr_run = 0
print "Finito!"

In [ ]:
print "Attendi..."
if load_from_npz:
    loaded = np.load(in_path + in_file)
    time_quantum = loaded['time_quantum']
    time_start = loaded['time_start']
    time_end = loaded['time_end']
    times = loaded['times']
    mean_throughput = loaded['mean_throughput']
    delta_throughput = loaded['delta_throughput']
    avg_throughput = loaded['avg_throughput']
    mean_delay = loaded['mean_delay']
    delta_delay = loaded['delta_delay']
    avg_delay = loaded['avg_delay']
    curr_run = n_traces
    print "Finito!"
    print "Dati caricati da " + in_path + in_file
else:
    in_file = os.listdir(in_path)
    print "Finito!"
    if (len(in_file) != n_traces):
        print "Il numero di tracce non corrisponde! Controllare il percorso di input!"
    else:
        print "Trovate " + str(len(in_file)) + " tracce da analizzare!"

In [ ]:
if load_from_npz:
    print "Eseguire la cella 2 per caricare i dati in memoria!"
elif (len(in_file) != n_traces):
    print "Il numero di tracce non corrisponde! Controllare il percorso di input!"
elif (curr_run == n_traces):
    print "Tutte le tracce sono state analizzate!"
else:
    print "Run: " + str(curr_run) 
    print "File di input: " + in_file[curr_run]
    print "Attendi..."
    
    curr_throughput = np.zeros((n_ues, len(times)))
    curr_delay = np.zeros((n_ues, len(times)))
    count = np.zeros((n_ues, len(times)))
    
    with open(in_path + in_file[curr_run], 'r') as f: 
        prev_line = ""
        for line in f:
            # elimino duplicati consecutivi
            if (line != prev_line):
                prev_line = '%s' % line
                tokens = line.rstrip().split(' ')
                # individuo i pacchetti ricevuti
                if (tokens[0] == "Rx"):
                    lcid = int(tokens[5])
                    # scarto i pacchetti di controllo
                    if (lcid > 1):
                        time = float(tokens[1])
                        if (time >= time_start) and (time < time_end):
                            # estraggo le informazioni
                            ue = int(tokens[3]) - 1
                            pck_size = float(tokens[6]) / (10e6 * time_quantum)
                            pck_delay = float(tokens[7]) / 10e6
                            slot = int(floor((time - time_start) / time_quantum))
                            # accumulo i valori di entrambi i tipi
                            mean_throughput[ue, slot] += pck_size / n_traces 
                            # accumulo i valori della traccia corrente
                            curr_throughput[ue, slot] += pck_size
                            curr_delay[ue, slot] += pck_delay
                            count[ue, slot] += 1
        f.close()
    # completo il calcolo della media
    count = count.clip(min = 1)
    curr_delay = np.divide(curr_delay, count)
    # accumulo i valori cross traccia
    power_throughput += np.divide(np.square(curr_throughput), n_traces)
    mean_delay += np.divide(curr_delay, n_traces)
    power_delay += np.divide(np.square(curr_delay), n_traces)

    curr_run += 1                    
    print "Finito!"
    print "Bisogna eseguire ancora " + str(n_traces - curr_run) + " run!"                          

In [ ]:
if load_from_npz:
    print "Eseguire la cella 2 per caricare i dati in memoria!"
elif (len(in_file) != n_traces):
    print "Il numero di tracce non corrisponde! Controllare il percorso di input!"
elif (curr_run != n_traces):
    print "Bisogna eseguire ancora " + str(n_traces - curr_run) + " run!"
else:
    print "Attendi..."
    # calcolo lo scarto quadratico medio con la formula sqrt(E(x^2) - E(x)^2)
    sigma_throughput = np.sqrt(power_throughput - np.square(mean_throughput))
    sigma_delay = np.sqrt(power_delay - np.square(mean_delay))
    
    # calcolo delta per l'intervallo di confidenza al 95%
    delta_throughput = 1.96 * np.divide(sigma_throughput, np.sqrt(n_traces) * np.ones(len(times)))
    delta_delay = 1.96 * np.divide(sigma_delay, np.sqrt(n_traces) * np.ones(len(times)))

    # calcolo la media temporale
    avg_throughput = np.mean(mean_throughput, axis = 1)
    avg_delay = np.mean(mean_delay, axis = 1)
    print "Finito!"

In [ ]:
if (not load_from_npz) and (len(in_file) != n_traces):
    print "Il numero di tracce non corrisponde! Controllare il percorso di input!"
elif (curr_run != n_traces):
    print "Bisogna eseguire ancora " + str(n_traces - curr_run) + " run!"
else:
    # plot CI
    for i in range(n_ues):
        fig = plt.figure(figsize = (20, 10))
        ax = plt.gca()
        ax.set_xlim(time_start, time_end)
        ax.xaxis.set_ticks(times)
        ax.xaxis.set_ticklabels(times)

        plt.subplot(2, 1, 1)
        plt.title("Throughput ue" + str(i + 1))
        plt.xlabel("Time [s]")
        plt.ylabel('Througput [Mb/s]')
        plt.grid(True)

        plt.plot(times, mean_throughput[i], lw = 2, color = 'b')
        plt.plot(times, mean_throughput[i] + delta_throughput[i], '--', lw = 1, color = 'b')
        plt.plot(times, avg_throughput[i] * np.ones(len(times)), '--', lw = 1, color = 'g')
        plt.plot(times, mean_throughput[i] - delta_throughput[i], '--', lw = 1, color = 'b')

        plt.legend(['throughput', 'CI 95%', "mean"])

        plt.subplot(2, 1, 2)
        plt.title("Delay ue" + str(i + 1))
        plt.xlabel("Time [s]")
        plt.ylabel('Delay [ms]')
        plt.grid(True)

        plt.plot(times, mean_delay[i], lw = 2, color = '#800080')
        plt.plot(times, mean_delay[i] + delta_delay[i], '--', lw = 1, color = '#800080')
        plt.plot(times, avg_delay[i] * np.ones(len(times)), '--', lw = 1, color = 'r')
        plt.plot(times, mean_delay[i] - delta_delay[i], '--', lw = 1, color = '#800080')

        plt.legend(['delay', 'CI 95%', "mean"])

        plt.show()

In [ ]:
# eseguire per salvare i risultati su file npz
if (not load_from_npz) and (len(in_file) != n_traces):
    print "Il numero di tracce non corrisponde! Controllare il percorso di input!"
elif (curr_run != n_traces):
    print "Bisogna eseguire ancora " + str(n_traces - curr_run) + " run!"
else:
    print "Attendi..."
    # percorso di output
    out_path = "/home/luca/Scrivania/TLCnet_prj/ns3-mmwave/output8/npz/" # da impostare
    out_file = "out.npz" # da impostare
    np.savez_compressed(out_path + out_file, time_quantum = time_quantum, \
                                             time_start = time_start, \
                                             time_end = time_end, \
                                             times = times, \
                                             mean_throughput = mean_throughput, delta_throughput = delta_throughput, \
                                             avg_throughput = avg_throughput, \
                                             mean_delay = mean_delay, delta_delay = delta_delay, \
                                             avg_delay = avg_delay)
    print "Finito!"
    print "Dati salvati in " + out_path + out_file